In [7]:
import pandas as pd
import numpy as np
from xlsxwriter.utility import xl_rowcol_to_cell

# ----------------------------------------------FILL IN THE INFORMATION HERE-----------------------------------------------------
# start  is the beginning of the auditing period and end is the end of the monitoring period. Date format is (year,month,day).
start = pd.Timestamp(2020,1,1)
end = pd.Timestamp(2021,7,30)
# report_name will be the name of the excel file generated be sure to remember to include .xslx at the end
report_name = "DDC-Test_Court_Audit.xlsx"
# ----------------------------------------------DONT CHANGE ANYTHING BEYOND THIS POINT-----------------------------------------------------

# ------------------------------------------CASE LIST------------------------------------------------------------------------------------------

cl_data = pd.read_csv('caselist.csv', parse_dates = ["Admission Date"])

# clean data to needed data types
cl_data['Admission Date'] = pd.to_datetime(cl_data['Admission Date'])

# cast all blanks to "-1", format discharge date, reason for active 
cl_data["Age Began Using Drugs"] = cl_data["Age Began Using Drugs"].replace([0],-1)
cl_data["Age Began Using Alcohol"] = cl_data["Age Began Using Alcohol"].replace([0],-1)
cl_data["Discharge Date"] = cl_data["Discharge Date"].replace([np.nan], "NA")
cl_data = cl_data.fillna(-1)
cl_data['Discharge Reason']=cl_data['Discharge Reason'].mask(cl_data['Discharge Reason'].eq(-1),cl_data['Discharge Date'])
cl_data['Discharge Housing Status']=cl_data['Discharge Housing Status'].mask(cl_data['Discharge Housing Status'].eq(-1),cl_data['Discharge Date'])


#obscure SSN
cl_data.loc[cl_data['SSN'] != -1, 'SSN'] = "Redacted"


# delete uneeded rows  
cl_data.drop(["Age at Screening", "Primary Offense", "Drug Court Case Outcome", "Offense Category", "Offense Date", "Court Program Approach","Incident Offense","Prior Adjudications", "FDOC", "Current Age", "Days From Acceptance to SA Treatment", "Days From Arrest to SA Treatment", "Days In Jail","Days In Jail By Bench Warrant",
"Days In Jail By Drug Court Sanction", "SID", "Days In Jail By New Offense", "Days In Jail By Pre-Admission", "Days of 12 Step Program",  "Days Rearrested in Program",
"Drug Court Control Group","First Treatment Date", "History of IV Use","In SA Treatment Program At Screening","IV Drug User", "Marital Status at Screening",
"Medication Compliance", "Medical Insurance Status", "Number of 12 Step Program Meetings Attended","Number of Bench Warrants", "Number of Days in Drug Court",
"Days Rearrested Post Program","Blood Alcohol Content", "CaseStatus", "Zip", "Recommended Treatment Modality At Screening", "Charge Type", "Arrest Date", "Number of Days in Phase I", "Number of Days in Phase II", "Number of Days in Phase III", "Number of Days in Phase IV", "Number of Days in Phase IV",
"Number of Felonies", "Number of Incentives", "Number of Monitoring with Contact Made - No Violation", "Number of Overall Positive Drug Tests", "Number of Other Offense",
"Number of Misdemeanors", "Number of Sanctions", "Number of Status Offenses", "Number of Scheduled Drug Court Reviews", "Percent of Positive Drug Tests", "Positive Drug Test In Program",
"1st Positive Drug Test (In Program)", "Reoffend Status Offense In Program", "Reoffend Status Offense Post Program", "Retention Days", "Deceased Date", "Number Of Ancillary Sessions",
"Number of Sessions", "SA Early Intervention/Education", "Primary Offense Charge Code","SA Intensive Outpatient", "SA Outpatient", "SA Outpatient Detox", "SA Residential", "SA Sub Acute Detox",
"SA Treatment Contact Hours", "Qualifying Sentencing Score", "Rearrest Date in Program", "Rearrest Date Post Program", "Rearrest in Program", "Case Management/Support Coordination Days",
"Co-occurring Treatment Service Days", "MH Treatment Contact Hours", "MH - Doctor/Medication Review Hours", "MH - Assertive Community Treatment (ACT) Hours",
"MH - Case Management/Support Coordination Hours", "MH - Inpatient Hospitalization/Partial Day Hospitalization Hours", "MH - Co-occurring Treatment Services Hours",
 "MH - Therapy Services Hours", "ClientID",  "PersonGUID", "MH - Residential Hours", "MH - Crisis Residential/Intensive Crisis Stabilization Hours",
 "MH - Employment Services Hours", "MH - Community Based Services Hours",  "Total Number of Drug Tests", "Days From Referral To Screening","Days From Screening To Acceptance", "PlacedInFosterHome",
 "Sobriety Days", "Drug of Choice-Secondary", "Drug of Choice-Tertiary", "Ethnicity", "Improvement in Mental Health", "Improvement in Quality of Life", "Prior Substance Abuse",
"Substance Abuse Disorder at Screening", "Drivers License Status", "Convicted Offense Category in Program", "FullName", "CourtID","ADDRESS","CaseID", "Prior Subsance Abuse Treatment"],axis=1,inplace= True)

# choose date range for cohort
cl_data = cl_data[(cl_data['Admission Date']>= start) & (cl_data['Admission Date']<= end)]  
# find rows with blanks and drop duplicates
cl_report = cl_data[cl_data.values == -1]
cl_report.drop_duplicates(inplace=True)
# ------------------------------------------SCREENING REPORT- COURT INVOLVEMENT----------------------------------------------------------------------------------------
oci_data = oci_data = pd.read_csv('p_ScreeningExport_GetOtherCourtInvolvement.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
oci_data['ScreeningDate'] = pd.to_datetime(oci_data['ScreeningDate'])

# fill in blank rows with -1
oci_data = oci_data.fillna(-1)

# delete uneeded rows  
oci_data.drop(["ScreeningID","LastName","FirstName", "MiddleName","DOB", "OutstandingWarrantStatus", "ProbationStatus", "PendingCriminalOffenses", "PriorAdjudications", "PreConvictionNonDomestic",
"PreConvictionDomestic","PreCourtFailToAppear", "PreDrugCourtParticipation","PreDrugCourtParticipationOther"],axis=1,inplace= True)

# choose date range for cohort
oci_data =oci_data[(oci_data['ScreeningDate']>= start) & (oci_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
oci_report = oci_data[oci_data.values == -1]
oci_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- SCREENING AND ASSESSMENT----------------------------------------------------------------------------------------
sa_data = sa_data = pd.read_csv('p_ScreeningExport_GetScreeningAssessments.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
sa_data['ScreeningDate'] = pd.to_datetime(sa_data['ScreeningDate'])

# fill in blank rows with -1
sa_data = sa_data.fillna(-1)

# delete uneeded rows  
sa_data.drop(["ScreeningID","LastName","FirstName", "Gender", "Ethnicity", "MiddleName","DOB", "ScreeningCaseManager", "SecondaryDrugOfChoice", "TertiaryDrugOfChoice", "Comment",
"ViolenceRisk", "RecidivismRisk", "FailToAppearRisk", "CommunityNonCompRisk", "OverallRisk", "OverallRiskOther", "ViolenceRiskOther", "RecidivismRiskOther",
"FailToAppearRiskOther", "CommunityNonCompRiskOther", "CriminalRecidivismRisk", "FamilyParenting", "EducationEmployement" ,"PeerRelations", "SubstanceAbuse", "LeisureRecreation",
"PersonalityBehavior", "AttitudesOrientation", "AssessmentType" , "ToolAdministeredOther", "Score", "PrimaryDiagnosis", "SecondaryDiagnosis", "IVDrugUser", "TimingOfAdministration"],axis=1,inplace= True)

# choose date range for cohort
sa_data =sa_data[(sa_data['ScreeningDate']>= start) & (sa_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
sa_report = sa_data[sa_data.values == -1]
sa_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- SCREENING AND ASSESSMENT----------------------------------------------------------------------------------------

dx_data = dx_data = pd.read_csv('p_ScreeningExport_GetDiagnosis.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
dx_data['ScreeningDate'] = pd.to_datetime(dx_data['ScreeningDate'])

# fill in blank rows with -1
dx_data = dx_data.fillna(-1)

# delete uneeded rows  
dx_data.drop(["ScreeningID","LastName","FirstName", "MiddleName","DOB", "SecondaryDiagnosis","OtherDiagnosis1", "OtherDiagnosis2", "OtherDiagnosis3"],axis=1,inplace= True)

# choose date range for cohort
dx_data = dx_data[(dx_data['ScreeningDate']>= start) & (dx_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
dx_report = dx_data[dx_data.values == -1]
dx_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- RELATED PARTIES---------------------------------------------------------------------------------------

rp_data  = pd.read_csv('p_ScreeningExport_GetRelatedParties.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
rp_data['ScreeningDate'] = pd.to_datetime(rp_data['ScreeningDate'])

# fill in blank rows with -1
rp_data = rp_data.fillna(-1)

# delete uneeded rows  
rp_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName","RelatedPartyDOB","Gender", "Ethnicity","Address1", "City", "State", "InfantWithPrenatalSubstanceExposure",
 "LivingSituation", "ChildCurrentCustodyStatus", "CurrentChildSupport", "Issues", "Attorney"],axis=1,inplace= True)

# choose date range for cohort
rp_data =rp_data[(rp_data['ScreeningDate']>= start) & (rp_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
rp_report = rp_data[rp_data.values == -1]
rp_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- Veteran Info---------------------------------------------------------------------------------------

mi_data  = pd.read_csv('p_ScreeningExport_GetVeteranInfo.csv', parse_dates = ["ScreeningDate"])

# clean data to needed data types
mi_data['ScreeningDate'] = pd.to_datetime(mi_data['ScreeningDate'])

# fill in blank rows with -1
mi_data = mi_data.fillna(-1)

# delete uneeded rows  
mi_data.drop(["DOB","ScreeningID","LastName","FirstName", "MiddleName","Participant", "EnlistmentOrCommissioningDate", "MilitaryOccupationalSpecialty",
"DateofMilitaryDischarge", "MilitaryDischargeStatus", "DetailsOfDischarge", "MilitaryRank","HasServedInUSMilitary", "ConvictionsPriorToMilitaryService", "DDForm214OrOtherSource","ConvictionsPriorToMilitaryServiceHighestOffense",
"AwardsAndDecorations", "RankReduction", "MilitaryIncarceration", "DeployedAbroad", "CombatZone", "CombatZoneStationedOutcome", "WitnessorInvolvementNumberOfTimes", "PTSD", "PTSDDetails",
"TBI", "TBIDetails", "IED_HME", "IED_HME_Details", "MST","MSTDetails", "SentToVAVJOReferralDate", "VeteransAssociationOrGroupMembership","VeteransAssociationOrGroupMembershipDetails",
"ReceivingDisabilityCompensationFromtheVAPercentDisabled","UtilizingServicesFromTheVetCenter","ReceivingDisabilityCompensationFromtheVA", "UtilizingServicesFromTheVetCenterDetails",  "HasSuicideRisk"],axis=1,inplace= True)

# choose date range for cohort
mi_data =mi_data[(mi_data['ScreeningDate']>= start) & (mi_data['ScreeningDate']<= end)]  

# find rows with blank and remove duplicate rows
mi_report = mi_data[mi_data.values == -1]
mi_report.drop_duplicates(inplace=True)

# ------------------------------------------SCREENING REPORT- MILITARY REPORT---------------------------------------------------------------------------------------

mil_data  = pd.read_csv('MilitaryServiceReport.csv', parse_dates = ["AdmissionDate"])

# clean data to needed data types
mil_data['AdmissionDate'] = pd.to_datetime(mil_data['AdmissionDate'])

# fill in blank rows with -1
mil_data["MilitaryDischargeDate"] = mil_data["MilitaryDischargeDate"].replace([np.nan], "NA")
mil_data = mil_data.fillna(-1)
##### need to cross reference discharge date and reason######
mil_data['MilitaryDischargeReason']=mil_data['MilitaryDischargeReason'].mask(mil_data['MilitaryDischargeReason'].eq(-1),mil_data['MilitaryDischargeDate'])


# delete uneeded rows  
mil_data.drop(["CaseID","Clerk Case Number","ClientID","FullName", "Gender", "Race", "DOB", "CaseManager", "Judge", "CasePhase", "CaseStatus",  "MilitaryRank",
"Months Deployed Abroad", "Deployment Location", "Occupational Specialty", "Years of Service", "Suicide Risk", "Enlistment Date", "Conviction prior to Military", "Awards & Decorations",
"Discplinary Action/Rank Reduction", "Military Incarceration", "Number of Deployments abroad", "Conflict Eras of Service", "Number of Times Witness or Involvement", "Date Assessment Received from VA/VJO",
"Veterans Association Membership", "Disability Compensation from VA", "Vet Center Utilization", "Veteran Mentor Assigned", "Veteran Mentor Name", "Veteran Mentor Assigned Date"
],axis=1,inplace= True)

# choose date range for cohort
mil_data =mil_data[(mil_data['AdmissionDate']>= start) & (mil_data['AdmissionDate']<= end)]  

# find rows with blank and remove duplicate rows
mil_report = mil_data[mil_data.values == -1]
mil_report.drop_duplicates(inplace=True)

# --------------------------------------write report to excel-----------------------------------
writer = pd.ExcelWriter(report_name, engine= 'xlsxwriter')
cl_report.to_excel(writer,index=False, sheet_name= "caselist")
oci_report.to_excel(writer,index=False, sheet_name= "othercourtinvolvement")
sa_report.to_excel(writer,index=False, sheet_name= "screeningassessment")
dx_report.to_excel(writer,index=False, sheet_name= "diagnosis")
rp_report.to_excel(writer,index=False, sheet_name= "relatedparties")
mi_report.to_excel(writer,index=False, sheet_name= "screening-military")
mil_report.to_excel(writer,index=False, sheet_name= "military")
workbook = writer.book
redfill = workbook.add_format({'bg_color': '#FFC7CE'})
worksheet = writer.sheets['caselist']
worksheet.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet1 = writer.sheets['othercourtinvolvement']
worksheet1.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet2 = writer.sheets['screeningassessment']
worksheet2.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet3 = writer.sheets['diagnosis']
worksheet3.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet4 = writer.sheets['relatedparties']
worksheet4.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet5 = writer.sheets['screening-military']
worksheet5.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})
worksheet6 = writer.sheets['military']
worksheet6.conditional_format('A1:EP15000',{
    'type':     'cell',
    'criteria': '==',
    'value':    -1,
    'format':   redfill})




writer.save()
writer.close()
    




C:\Users\taylo\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\taylo\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\taylo\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*ar